# **MDP를 모를 때 최고의 정책 찾기**

MDP를 모를 때, 최고의 정책을 찾기 위해서 평가단계에서 MC를 활용하는 방법을 공부했는데, 

TD도 활용가능함

## **2. SARSA**

TD로 V 학습 $V(s) ← V(s) + \alpha(r + \gamma{V(s')} - V(s))$

TD로 Q 학습(SARSA) $Q(s,a) ← Q(s, a) + \alpha(r + \gamma{Q(s', a')} - Q(s, a))$

$$ SARSA = S → A → R → S' → A' $$


모두 벨만 기대방정식에서 나온 값

$$ v_\pi(s_t) = E_\pi[r_{t+1} + \gamma{v_{\pi}}(s_{t+1})]$$
$$ q_{\pi}(s_t, a_t) = E_\pi[r_{t+1} + \gamma{q_{\pi}}(s_{t+1}, a_{t+1})]$$

### **1) 개요**
챕터 4에서 MDP를 알 때 최적의 정책 함수를 찾는 방법론을 배웠음. 그 중 정책 이터레이션은 임의의 정책에서 시작하여 정책을 평가하고 밸류를 계산하고 그리디 정책을 만드는 과정을 계속해서 반복하는 방법이었는데, 지금 상황과의 차이는 MDP를 모른다는 점에서만 차이가 난다.


**1) 정책을 평가하기 위한 상태별 보상과 전이확률을 알 수 없다.**
$$v_\pi(s) = \sum_{a{\in}A}\pi(a|s)\biggl(r^a_s + \gamma\sum_{s'{\in}S}P^a_{ss'}v_{\pi}(s')\biggl)$$ 

$$ *여기서 전이확률과 보상을 모르는 상태 $$

 - 따라서 직접 액션을 해보기 전까지는 다음 상태가 어디일지, 얼마의 확률로 도착한 것인지, 보상은 얼마인지 알 수 없음

**2) 어떻게든 상태별 보상에 대한 평가를 마쳤더라도 정책 개선 단계에서 그리디한 정책을 만들 수 없다.**
 - 전이확률을 안다 치고 $s0$에서 $a1$과 , $a2$ 액션을 했을 때 어느 상태로 도착할 지 알 수 없기 때문에 그리디한 정책을 채택할 수 없음

### **2) 해결방법**

**1) 평가자리에 MC**

 - 각 상태의 밸류를 계산하기 위해 MC또는 TD를 활용한다.
    
**2) V 대신 Q**

 - 상태가치함수 $v(s)$만 가지고는 **정책개선**에서 그리디 정책을 생성할 수 없으니 $v(s)$대신 행동가치함수 $q(s,a)$를 활용
 - 상태 $s0$에서 액션 $a1$, $a2$ 액션을 했을 때 상태 $s1$, $s2$로 전이된 $v(s1)$, $v(s2)$를 구하는 것이 아니라 $q(s0, a1)$, $q(s0, a2)$를 화룡한다는 의미
 - 이를 활용하면 $q(s0, a1) = 1$, $q(s0, a2) = 2$를 알 수 있기 때문에 그리디 액션을 선택할 수 있음

**3) greedy 대신 decaying $\epsilon$-greedy(내가 아는 그 입실론 그리디)**
여기서 갑자기 $\epsilon$-greedy가 튀어나온 이유는 무엇인가?
 - MDP를 모르는 상황이기 때문에 MC를 이용하여 q(s,a)를 계산하고, 그리디 정책을 만들고, 다시 MC를 이용해 q(s,a)를 계산하는 과정을 반복하게 된다.
 - 문제는 그리디한 정책하에서 한 번 q(s0,a1) = 0.1로 계산하면, 다른 액션을 했을 때q(s0,a2)=0(모르니까 0임, 초기화)이기 때문에 무조건 q(s0,a1)만 선택하게 된다.
 - 따라서 $\epsilon$을 부여해서 탐색의 정도를 설정하는 것
 $$\pi(a|s) = \begin{cases}1 - \epsilon& \text{if a* = argmax_a q(s,a)}\\
            \epsilon &\text{otherwise}\end{cases}$$

    - 만약 $\epsilon$을 0.1로 가정하면, 90%확률로 $q(s,a)$가 높은 액션을 선택하고, 10%의 확률로 랜덤하게 액션을 선택한다.(탐색과 활용)
    - 단, 학습 초기에는 $\epsilon$을 높게 잡았다가 데이터가 쌓이게 되면 활용에 집중하기 위해 $\epsilon$을 낮춤

### **3) 몬테카를로 컨트롤 구현**

**MCC 요약**

 1) 간략화된 정책 이터레이션을 활용함(정책 평가와 개선을 수렴할 때까지 진행하지 않음, 챕터 4 참고)
 
 2) 한 에피소드의 경험을 쌓고
 
 3) (정책평가) 경험한 데이터로 $q(s,a)$ 테이블의 값을 업데이트하고
 
 4) (정책개선) 업데이트된 $q(s,a)$ 테이블을 이용하여 $\epsilon$-greedy 정책을 만들고
 
 5) (반복) 이를 반복한다.

In [1]:
import pandas as pd
pd.DataFrame([['길','길','벽','길','길','길','길'], 
              ['길','길','벽','길','길','길','길'],
              ['시작','길','벽','길','벽','길','길'],
              ['길','길','길','길','벽','길','길'],
             ['길','길','길','길','벽','길','종료']])

,0,1,2,3,4,5,6
0,길,길,벽,길,길,길,길
1,길,길,벽,길,길,길,길
2,시작,길,벽,길,벽,길,길
3,길,길,길,길,벽,길,길
4,길,길,길,길,벽,길,종료


In [2]:
from ch6_SARSA import GridWorld, QAgent

## QAgent 클래스

In [17]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # q벨류를 저장하는 변수. 모두 0으로 초기화. (5개의 7 * 4 그리드월드)
        self.eps = 0.9

    def select_action(self, s):
        # eps-greedy로 액션을 선택해준다
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition): # MCC에서는 transition의 history 전체를 input으로 받았는데, SARSA에서는 TD를 활용하다보니 transition을 받는다.
        s, a, r, s_prime = transition
        x,y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) # S'에서 선택할 액션 (실제로 취한 액션이 아님)
        # SARSA 업데이트 식을 이용
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1 * (r + self.q_table[next_x,next_y,a_prime] - self.q_table[x,y,a])

    def anneal_eps(self):
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)

    def show_table(self):
        q_lst = self.q_table.tolist()
        data = np.zeros((5,7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

## main함수

In [6]:
def main():
    env = GridWorld()
    agent = QAgent()

    for n_epi in range(1000):
        done = False

        s = env.reset()
        while not done:
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s,a,r,s_prime))
            s = s_prime
        agent.anneal_eps()

    agent.show_table()


if __name__ == '__main__':
    main()

[[3. 3. 0. 2. 2. 2. 3.]
 [3. 3. 0. 2. 2. 3. 3.]
 [2. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [2. 2. 2. 3. 0. 2. 0.]]


In [7]:
t = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]
# t[1][1]
for row in t:
    print(row)

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 16]


In [15]:
t = [1, 2, 3, 4, 5]
t[::-1]

[5, 4, 3, 2, 1]